In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTMLinteractions_df
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import time

pd.set_option('display.max_columns', 50)

from evaluation_and_testing.testing import evaluate_train_test_split_explicit
from evaluation_and_testing.testing import evaluate_leave_one_out_explicit
from evaluation_and_testing.testing import evaluate_train_test_split_implicit
from evaluation_and_testing.testing import evaluate_leave_one_out_implicit

from recommenders.recommender import Recommender

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [4]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
print(len(ml_ratings_df['item_id'].unique()))
print(len(ml_movies_df['item_id'].unique()))


display(ml_movies_df)
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
print(len(ml_df['item_id'].unique()))
display(ml_ratings_df.loc[(ml_ratings_df['item_id'] == 150)])

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of chosen interactions: {}".format(len(ml_ratings_df)))

display(ml_df.head(10))

9724
9742


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


9724


,user_id,item_id,rating,timestamp
524,5,150,3.0,847434748
618,6,150,4.0,845553110
877,7,150,4.5,1106635925
1037,8,150,4.0,839463422
1265,11,150,5.0,902154266
...,...,...,...,...
95990,602,150,4.0,840875508
97148,605,150,5.0,1277096909
98487,607,150,5.0,963079343
98694,608,150,2.0,1117162618


Number of chosen interactions: 9692


,user_id,item_id,rating,timestamp,title,genres
572,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
573,5,50,4.0,847434881,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
574,6,50,1.0,845553381,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
575,7,50,4.5,1106635993,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
576,8,50,5.0,839463644,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
577,16,50,4.0,1377476781,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
578,17,50,4.5,1305697013,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
579,18,50,5.0,1455049343,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
580,23,50,4.0,1107163741,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
581,24,50,4.0,1458942023,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


**Task 1.** Implement the MostPopularRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [78]:
class MostPopularRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        
        # Zmienna przechowująca model
        self.model = None
        
        # Zmienna MultiLabelBinarizera
        self.mlb = None
        
        # Zmienna przechowująca słownik powstały z DataFrame'u users_df
        # klucz: user (identyfikator) 
        # wartość: lista wartości będąca rozkładem (procentowym) mówiącym jaką część wszystkich obejrzanych filmów użytkownika
        # stanowią filmy należące do danego gatunku
        self.users_dict = None
        
        # Nazwy kolumn (gatunki filmów) wykorzystywane w DataFrame users_df
        self.users_features = None
        
        # Wykorzystywanie iloczynu sklaranego między rozkładem użytkownika i rozkładem filmu
        self.dot_product = True
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        ml_df = pd.merge(interactions_df, items_df, on='item_id')
        ml_df['popularity'] = 1
        movie_popularity = ml_df.loc[:, ['item_id', 'popularity']].groupby('item_id').sum().sort_values(by='popularity', ascending=False).reset_index()
        #display(movie_popularity.head(30))
        movie_popularity = pd.merge(movie_popularity, items_df, on='item_id')
        self.model = movie_popularity
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        #display(self.model.head(10))
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for idx, user in users_df.iterrows():
            user_recommendations = []
            for i in range(n_recommendations):
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': self.model.iloc[i]['item_id'],
                        'score': 1.0
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])
        
        return recommendations
        
    
# Quick test of the recommender

most_popular_recommender = MostPopularRecommender()
most_popular_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = most_popular_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)
#display(recommendations)
recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
#display(HTML(recommendations.to_html()))

most_popular_recommender = MostPopularRecommender()

most_popular_results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    most_popular_recommender, ml_ratings_df, ml_movies_df, max_evals=300, seed=6789))]

most_popular_results = pd.DataFrame(
    most_popular_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(most_popular_results.to_html()))

most_popular_recommender = MostPopularRecommender()

most_popular_results = [['MostPopularRecommender'] + list(evaluate_train_test_split_implicit(
    most_popular_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

most_popular_results = pd.DataFrame(
    most_popular_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(most_popular_results.to_html()))
display(most_popular_recommender.model)

Recommendations


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.076063,0.284116,0.440716,0.680089,0.076063,0.192392,0.256306,0.332798


,item_id,popularity,title,genres
0,50,170,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
1,780,160,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
2,858,141,"Godfather, The (1972)",Crime|Drama
3,595,112,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
4,1704,105,Good Will Hunting (1997),Drama|Romance
...,...,...,...,...
915,55272,1,We Own the Night (2007),Crime|Drama
916,6245,1,Sweet Charity (1969),Comedy|Drama|Musical|Romance
917,54910,1,Behind the Mask: The Rise of Leslie Vernon (2006),Comedy|Horror|Thriller
918,54768,1,Daddy Day Camp (2007),Children|Comedy


In [5]:
ml_df_copy = ml_df.copy()
display(ml_df_copy.head(10))
ml_df_copy['val'] = 1
movie_popularity = ml_df_copy.loc[:,['item_id', 'val']].groupby('item_id').sum().sort_values(by='val', ascending=False).reset_index()

#display(ml_df_copy.loc[(ml_df_copy['item_id'] == 4)])

,user_id,item_id,rating,timestamp,title,genres
572,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
573,5,50,4.0,847434881,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
574,6,50,1.0,845553381,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
575,7,50,4.5,1106635993,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
576,8,50,5.0,839463644,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
577,16,50,4.0,1377476781,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
578,17,50,4.5,1305697013,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
579,18,50,5.0,1455049343,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
580,23,50,4.0,1107163741,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
581,24,50,4.0,1458942023,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


**Task 2.** Implement the HighestRatedRecommender (check the slides for class 1), but recommend only those movies which got at least 50 ratings. Evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [8]:
class HighestRatedRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        
        # Zmienna przechowująca model
        self.model = None
        
        # Zmienna MultiLabelBinarizera
        self.mlb = None
        
        # Zmienna przechowująca słownik powstały z DataFrame'u users_df
        # klucz: user (identyfikator) 
        # wartość: lista wartości będąca rozkładem (procentowym) mówiącym jaką część wszystkich obejrzanych filmów użytkownika
        # stanowią filmy należące do danego gatunku
        self.users_dict = None
        
        # Nazwy kolumn (gatunki filmów) wykorzystywane w DataFrame users_df
        self.users_features = None
        
        # Wykorzystywanie iloczynu sklaranego między rozkładem użytkownika i rozkładem filmu
        self.dot_product = True
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df['popularity'] = 1
        #display(ml_df.head(30))
        movie_popularity = interactions_df.loc[:, ['item_id', 'rating', 'popularity']].groupby('item_id').agg({'rating': 'mean', 'popularity': 'sum'}).sort_values(by='rating', ascending=False)
        movie_popularity = movie_popularity.loc[movie_popularity['popularity'] >= 50].reset_index()
        #movie_popularity = pd.merge(movie_popularity, items_df, on='item_id')
        self.model = movie_popularity
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for idx, user in users_df.iterrows():
            user_recommendations = []
            for i in range(n_recommendations):
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': int(self.model.iloc[i]['item_id']),
                        'score': self.model.iloc[i]['rating']
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations]).reset_index(drop=True)
        
        return recommendations
    
# Quick test of the recommender

highest_rated_recommender = HighestRatedRecommender()
highest_rated_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = highest_rated_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

highest_rated_recommender = HighestRatedRecommender()

highest_rated_results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
    highest_rated_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]


highest_rated_results = pd.DataFrame(
    highest_rated_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(highest_rated_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,858,4.289062,"Godfather, The (1972)",Crime|Drama
1,1,750,4.268041,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Comedy|War
2,1,1213,4.250000,Goodfellas (1990),Crime|Drama
3,1,50,4.237745,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
4,1,1225,4.184211,Amadeus (1984),Drama
5,1,1203,4.149123,12 Angry Men (1957),Drama
6,1,2502,4.090426,Office Space (1999),Comedy|Crime
7,1,1704,4.078014,Good Will Hunting (1997),Drama|Romance
8,1,2692,4.000000,Run Lola Run (Lola rennt) (1998),Action|Crime
9,1,6502,3.974138,28 Days Later (2002),Action|Horror|Sci-Fi


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.013333,0.04,0.073333,0.113333,0.013333,0.02754,0.041603,0.054395


**Task 3.** Implement the RandomRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [1]:
class RandomRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self, seed=0):
        """
        Initialize base recommender params and variables.
        
        :param int seed: Seed for the random number generator.
        """
        
        # Zmienna przechowująca model
        self.model = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        movie_ids = list(pd.merge(interactions_df, items_df, on='item_id')['item_id'].unique())
        self.model = movie_ids
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for idx, user in users_df.iterrows():
            
            user_recommendations = []

            random_movie_ids = self.rng.choice(self.model, size=n_recommendations)
            
            for idi in random_movie_ids:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': idi,
                        'score': 1.0
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])
        
        return recommendations
    
# Quick test of the recommender

random_recommender = RandomRecommender()
random_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = random_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = recommendations.merge(ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

random_recommender = RandomRecommender(seed=seed)

# random_results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
#     random_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=seed))]
random_results = [['RandomRecommender'] + list(evaluate_train_test_split_implicit(recommender, ml_ratings_df, ml_movies_df))]

random_results = pd.DataFrame(
    random_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(random_results.to_html()))

NameError: name 'Recommender' is not defined

# Linear Regression Recommender

For every movie we transform its genres into one-hot encoded features and we normalize them, for every user we count percentages for all genres how often do they appear among films watched by the user, we multiply both vectors (for the item and the user) to obtain explanatory variables, and then we fit a linear regression model to those features and actual ratings.

**Task 4.** Implement the calculate_user_genres method for calculating a DataFrame with one row per user and columns corresponding to genres (e.g. 'user_action', 'user_drama') with values calculated as follows:

- count the number of times a given user watched a given genre,
- apply a natural logarithm to this value plus one,
- normalize those values so that the sum of them over all columns within a row is equal to 1.

Implement the calculate_item_genres method for replacing the 'genres' column with one column per genre (e.g. 'action', 'drama') with values calculated as follows:

- place 1 in every column for which the genre appears in genres,
- normalize those values so that the sum of them over all genre columns within a row is equal to 1.

If item_features is None, then first find all genres and prepare a list of them. If item_features is not None, then create columns based on this list. Return both the transformed DataFrame and the list of genres.  Do not use MultiLabelBinarizer.

Note that in this second method you have preserve.

Evaluate the LinearRegressionRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [8]:
# Prepare data for tests
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")

In [116]:
def calculate_user_genres(interactions_df):
    users_df = interactions_df.copy()[['user_id', 'genres']].explode('genres')
    users_df['val'] = 1
    users_df = pd.pivot_table(users_df, index='user_id', columns='genres', values='val', aggfunc='count', fill_value=0.0).add_prefix("user_")
    users_df.columns.name = None
    users_df = users_df.applymap(lambda x: np.log(x+1))
    users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
    return users_df

In [117]:
# Test the calculate_user_genres method
display(calculate_user_genres(interactions_df))

,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western
user_id,,,,,,,,,,,,,,,,,,,,
1,0.102311,0.093749,0.059149,0.076449,0.093749,0.083027,0.000000,0.068670,0.059149,0.000000,0.000000,0.000000,0.029575,0.029575,0.0,0.076449,0.068670,0.083027,0.046875,0.029575
2,0.144386,0.000000,0.000000,0.000000,0.000000,0.144386,0.000000,0.255742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.091097,0.091097,0.182194,0.000000,0.091097
3,0.256885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162077,0.000000,0.162077,0.000000,0.000000,0.000000,0.0,0.000000,0.256885,0.162077,0.000000,0.000000
4,0.035810,0.035810,0.056758,0.071621,0.128379,0.092569,0.000000,0.143242,0.056758,0.035810,0.000000,0.035810,0.071621,0.000000,0.0,0.107431,0.000000,0.056758,0.035810,0.035810
5,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.000000,0.133333,0.066667,0.000000,0.000000,0.066667,0.066667,0.066667,0.0,0.133333,0.000000,0.066667,0.066667,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.065798,0.060970,0.029090,0.047444,0.096320,0.073415,0.018354,0.111728,0.063494,0.018354,0.036708,0.029090,0.018354,0.047444,0.0,0.089163,0.055062,0.073415,0.047444,0.018354
607,0.113226,0.075484,0.000000,0.037742,0.105955,0.087634,0.000000,0.130566,0.037742,0.000000,0.075484,0.000000,0.037742,0.037742,0.0,0.059820,0.087634,0.075484,0.037742,0.000000
608,0.085929,0.067345,0.045723,0.070753,0.096567,0.075138,0.000000,0.083142,0.067345,0.000000,0.065454,0.028035,0.028035,0.041071,0.0,0.061191,0.075138,0.081099,0.028035,0.000000


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

def calculate_item_genres(item_genres, item_features=None):
    
    item_genres = item_genres[['item_id', 'genres']]
    item_genres = item_genres.explode('genres')
    item_genres = pd.get_dummies(item_genres.explode('genres'), prefix="", prefix_sep="").drop_duplicates()
    item_genres = item_genres.groupby('item_id').sum().fillna(0)
    cols = item_genres.columns.values.tolist()
    
    if item_features == None:
        item_genres = item_genres / item_genres.sum(axis=1).values.reshape(-1, 1)
        item_features = cols
    else:
        diff1 = list(set(item_features) - set(cols))
        item_genres[diff1] = 0
        diff2 = list(set(cols).intersection(set(item_features)))
        item_genres = item_genres.loc[:, diff1 + diff2]
        item_genres = item_genres / item_genres.sum(axis=1).values.reshape(-1, 1)
        item_genres = item_genres.fillna(0)

    return item_genres, item_features

In [12]:
# # Test the method
# #display(sorted(list(interactions_df['item_id'].unique())))
item_genres, item_features = calculate_item_genres(interactions_df)
display(item_genres)
print(item_features)

item_genres, item_features = calculate_item_genres(interactions_df, ['comedy', 'crime', 'drama', 'horror', 'test_category'])
display(item_genres)
print(item_features)

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
item_id,,,,,,,,,,,,,,,,,,,,
4,0.000000,0.00,0.0,0.00,0.333333,0.000000,0.0,0.333333,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.00,0.000000,0.0,0.0
12,0.000000,0.00,0.0,0.00,0.500000,0.000000,0.0,0.000000,0.00,0.0,0.5,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0
50,0.000000,0.00,0.0,0.00,0.000000,0.333333,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.00,0.333333,0.0,0.0
74,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.0,0.500000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.500000,0.00,0.000000,0.0,0.0
78,0.250000,0.00,0.0,0.00,0.000000,0.250000,0.0,0.250000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.250000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184987,0.000000,0.25,0.0,0.25,0.000000,0.000000,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.0
185473,0.000000,0.00,0.0,0.00,1.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0
189381,0.333333,0.00,0.0,0.00,0.000000,0.333333,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.333333,0.0,0.0


['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'imax', 'musical', 'mystery', 'no_genres_listed', 'romance', 'sci_fi', 'thriller', 'war', 'western']


,test_category,comedy,drama,horror,crime
item_id,,,,,
4,0.0,0.5,0.5,0.0,0.0
12,0.0,0.5,0.0,0.5,0.0
50,0.0,0.0,0.0,0.0,1.0
74,0.0,0.0,1.0,0.0,0.0
78,0.0,0.0,0.5,0.0,0.5
...,...,...,...,...,...
184987,0.0,0.0,0.0,0.0,0.0
185473,0.0,1.0,0.0,0.0,0.0
189381,0.0,0.0,0.0,0.0,1.0


['comedy', 'crime', 'drama', 'horror', 'test_category']


In [118]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

from recommenders.recommender import Recommender

class LinearRegressionRecommender(Recommender):
    """
    Linear regression recommender class.
    """
    
    def __init__(self):
        """
        Initialize recommender params and variables.
        """
        self.model = None
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        
        self.uses_dot_product = True
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Transform genres to a more code-friendly form
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
        
        # Prepare user features
        
        users_df = interactions_df[['user_id', 'genres']].copy()
        users_df = users_df.explode('genres')
        users_df['val'] = 1
        users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
        users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
        users_df = users_df.rename_axis(None, axis=1).fillna(0)
        users_df = users_df.add_prefix('user_')
#         display(users_df.head(10))
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
#         display(interactions_df.head(10))
                
        # Prepare item features
        
        # Transform genres into binary values
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        # Normalize the values so that each movie's genres sum up to 1
        
        interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
            / interactions_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)
        
#         display(interactions_df.loc[:, self.mlb.classes_].head(10))

        # Prepare input data and fit the model
    
        # Dot product for personalization
        if self.uses_dot_product:
            interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
                * interactions_df[self.user_features].values
    
#         display(interactions_df.head(10))
        
        if self.uses_dot_product:  # Personalized
            x = interactions_df.loc[:, self.mlb.classes_].values
        else:  # Non-personalized
            x = interactions_df.loc[:, list(self.mlb.classes_) + self.user_features].values
        
        y = interactions_df['rating'].values
    
        self.model = LinearRegression().fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Transform the item to be scored into proper features
        
        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        items_df[self.mlb.classes_] = items_df[self.mlb.classes_] \
            / items_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
                user_df.columns = self.user_features
#             display(user_df)
#             display(items_df)
            input_df = items_df.copy()
            
            if self.uses_dot_product:
                input_df[self.mlb.classes_] = items_df[self.mlb.classes_] * user_df.values
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, self.mlb.classes_].values)
            else:
                input_df = input_df.merge(user_df, how='cross')
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, list(self.mlb.classes_) + self.user_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
    def _transform_genres(self, df):
        """
        Transforms a string with genres into a list of cleaned genre names.
        
        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df

In [119]:
# Quick test of the recommender

lr_recommender = LinearRegressionRecommender()
lr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = lr_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,title,genres
0,3,51562,3.501376,Babylon 5: The Gathering (1993),Sci-Fi
1,3,5468,3.501376,20 Million Miles to Earth (1957),Sci-Fi
2,3,176371,3.501376,Blade Runner 2049 (2017),Sci-Fi
3,3,147384,3.501376,Doctor Who: The Runaway Bride (2007),Sci-Fi
4,3,2698,3.501376,Zone 39 (1997),Sci-Fi
5,3,2661,3.501376,It Came from Outer Space (1953),Sci-Fi
6,3,2311,3.501376,2010: The Year We Make Contact (1984),Sci-Fi
7,3,3926,3.425357,Voyage to the Bottom of the Sea (1961),Adventure|Sci-Fi
8,3,4953,3.425357,"People That Time Forgot, The (1977)",Adventure|Sci-Fi
9,3,3994,3.425357,Unbreakable (2000),Drama|Sci-Fi


### Train-test split test

In [15]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_train_test_split_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.016594,0.348461,0.230529


Total evaluation time: 27.317997455596924


### Leave-one-out test

In [16]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_leave_one_out_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.045415,0.401399,0.242301


Total evaluation time: 37.95081686973572


# TF-IDF Recommender
TF-IDF stands for term frequency–inverse document frequency. Typically Tf-IDF method is used to assign keywords (words describing the gist of a document) to documents in a corpus of documents.

In our case we will treat users as documents and genres as words.

Term-frequency is given by the following formula:
<center>
$$
    \text{tf}(g, u) = f_{g, u}
$$
</center>
where $f_{g, i}$ is the number of times genre $g$ appear for movies watched by user $u$.

Inverse document frequency is defined as follows:
<center>
$$
    \text{idf}(g) = \log \frac{N}{n_g}
$$
</center>
where $N$ is the number of users and $n_g$ is the number of users with $g$ in their genres list.

Finally, tf-idf is defined as follows:
<center>
$$
    \text{tfidf}(g, u) = \text{tf}(g, u) \cdot \text{idf}(g)
$$
</center>

In our case we will measure how often a given genre appears for movies watched by a given user vs how often it appears for all users. To obtain a movie score we will take the average of its genres' scores for this user.

**Task 5.** Implement the following method for calculating a TF-IDF scores in a form of a dict (use defaultdict):

`{(1, 'action'): 0.45306430692185395, (1, 'adventure'): 0.39370003643934415, (1, 'animation'): 0.20886242957049514, ...}`

without using TfidfVectorizer (you can use loops).

Evaluate the TFIDFRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [85]:
def calculate_tf_idf_scores(interactions_df):
    tfidf_scores = defaultdict(lambda: 0.0)
    interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
    users_df = interactions_df.copy()[['user_id', 'genres']].explode('genres')
    users_df['val'] = 1
    
    tf = pd.pivot_table(users_df, index='user_id', columns='genres', values='val', aggfunc='sum', fill_value=0.0)
    tf.columns.name = None
    
    N = len(tf)

    ng = np.count_nonzero(tf, axis=0)
    ng = pd.DataFrame([ng], columns=tf.columns.values.tolist())

    idf = ng.applymap(lambda x: np.log(N/x))
    
    for u in tf.index.values.tolist():
            for g in list(tf.columns.values):
                tfidf_scores[(u, g)] = (tf.loc[u][g] * idf[g])[0]
    
    return tfidf_scores

In [86]:
# Test the method
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()     
tfidf_scores = calculate_tf_idf_scores(interactions_df)
tfidf_scores[(1, 'action')]

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
user_id,,,,,,,,,,,,,,,,,,,,
1,10,8,3,5,8,6,0,4,3,0,0,0,1,1,0,5,4,6,2,1
2,2,0,0,0,0,2,0,6,0,0,0,0,0,0,0,1,1,3,0,1
3,2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,2,1,0,0
4,1,1,2,3,11,5,0,15,2,1,0,1,3,0,0,7,0,2,1,1
5,0,0,1,1,1,1,0,3,1,0,0,1,1,1,0,3,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,11,9,2,5,37,15,1,67,10,1,3,2,1,5,0,28,7,15,5,1
607,7,3,0,1,6,4,0,10,1,0,3,0,1,1,0,2,4,3,1,0
608,28,13,5,15,43,18,0,25,13,0,12,2,2,4,0,10,18,23,2,0


,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
0,0.219805,0.325165,0.876138,0.62215,0.22398,0.384778,2.586928,0.171014,0.550046,3.061386,1.075471,0.864162,0.929759,0.647388,5.294978,0.306816,0.318245,0.284343,1.060872,1.178655


2.198046500486951

In [55]:
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
#         print("Training set")
#         display(interactions_df.head(10))
#         print()
#display(interactions_df.loc[(interactions_df['user_id'] == 1)])
user_genres = interactions_df.loc[:, ['user_id', 'genres']]
user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace("-", "_", regex=False)
user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace(" ", "_", regex=False)
display(interactions_df.loc[(interactions_df['user_id'] == 1)])
display(user_genres.loc[(user_genres['user_id'] == 1)])
user_genres = user_genres.groupby('user_id').aggregate(lambda x: "|".join(x))
user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace("|", " ", regex=False)
print(user_genres['genres'].loc[1])

,user_id,item_id,rating,timestamp,title,genres
0,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
204,1,423,3.0,964982363,Blown Away (1994),Action|Thriller
214,1,543,4.0,964981179,So I Married an Axe Murderer (1993),Comedy|Romance|Thriller
255,1,780,3.0,964984086,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
457,1,940,5.0,964982176,"Adventures of Robin Hood, The (1938)",Action|Adventure|Romance
465,1,1213,5.0,964982951,Goodfellas (1990),Crime|Drama
591,1,1500,4.0,964980985,Grosse Pointe Blank (1997),Comedy|Crime|Romance
650,1,1777,4.0,964981230,"Wedding Singer, The (1998)",Comedy|Romance
706,1,2000,4.0,964982211,Lethal Weapon (1987),Action|Comedy|Crime|Drama
781,1,2046,4.0,964982563,Flight of the Navigator (1986),Adventure|Children|Sci-Fi


,user_id,genres
0,1,Crime|Mystery|Thriller
204,1,Action|Thriller
214,1,Comedy|Romance|Thriller
255,1,Action|Adventure|Sci_Fi|Thriller
457,1,Action|Adventure|Romance
465,1,Crime|Drama
591,1,Comedy|Crime|Romance
650,1,Comedy|Romance
706,1,Action|Comedy|Crime|Drama
781,1,Adventure|Children|Sci_Fi


Crime Mystery Thriller Action Thriller Comedy Romance Thriller Action Adventure Sci_Fi Thriller Action Adventure Romance Crime Drama Comedy Crime Romance Comedy Romance Action Comedy Crime Drama Adventure Children Sci_Fi Animation Children Comedy Musical Adventure Animation Children Fantasy Comedy Crime Action Crime Action Comedy Sci_Fi Action Adventure Thriller Drama War Action Drama War Action Children Fantasy Adventure Fantasy Romance Action Adventure Thriller Comedy Western Adventure Animation Children Sci_Fi


In [ ]:
class TFIDFRecommender(Recommender):
    """
    Recommender based on the TF-IDF method.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.tfidf_scores = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        self.tfidf_scores = defaultdict(lambda: 0.0)

        # Prepare the corpus for tfidf calculation
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
                
        self.tfidf_scores = calculate_tf_idf_scores(interactions_df)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Transform genres to a unified form used by the vectorizer
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
                
        # Score items    
        
        for uix, user in users_df.iterrows():
            items = []
            for iix, item in items_df.iterrows():
                score = 0.0
                for genre in item['genres']:
                    score += self.tfidf_scores[(user['user_id'], genre)]
                score /= len(item['genres'])
                items.append((item['item_id'], score))
                
            items = sorted(items, key=lambda x: x[1], reverse=True)
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [item[0] for item in items][:n_recommendations],
                                                 'score': [item[1] for item in items][:n_recommendations]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [ ]:
# Print movies watched by user 3, 5, 39

active_user_movies = ml_df.loc[(ml_df['user_id'] == 3) | (ml_df['user_id'] == 5) | (ml_df['user_id'] == 39)]
print("Active users history")
display(active_user_movies)

In [ ]:
# Quick test of the recommender

tfidf_recommender = TFIDFRecommender()
tfidf_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = tfidf_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(recommendations)

### Train-test split test

In [ ]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_train_test_split_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

### Leave-one-out test

In [ ]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

**Task 6\*.** Implement an SVRRecommender by replacing the LinearRegression model with the SVR model (`from sklearn.svm import SVR`). Use separate features for users and items, not the dot product.

Tune params of the SVR model to obtain as good results as you can. 

To do tuning properly:

- divide the set into training, validation and test sets (randomly divide the dataset in proportions 60%-20%-20%),
- train the model with different sets of tunable parameters on the training set, 
- choose the best tunable params based on results on the validation set, 
- provide the final evaluation metrics on the test set for the best model obtained during tuning.

Recommended method of tuning: use hyperopt. Install the package using the following command: `pip install hyperopt`
    
Print the RMSE, MAPE and TRE on the test set. Use seed 6789.

In [5]:
from sklearn.svm import SVR
from sklearn.preprocessing import MultiLabelBinarizer

    
class SVRRecommender(Recommender):
    """
    SVR recommender class.
    """

    def __init__(self, kernel='rbf', C=1.0, epsilon=0.1):
        """
        Initialize base recommender params and variables.
        """
        super().__init__()
        self.model = None
        self.mlb = None
        self.kernel = kernel
        self.c = C
        self.epsilon = epsilon
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        self.uses_dot_product = False

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id
            and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id
            and the item feature columns.
        """

        # Transform genres to a more code-friendly form
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
        
        # Prepare user features
        
        users_df = interactions_df[['user_id', 'genres']].copy()
        users_df = users_df.explode('genres')
        users_df['val'] = 1
        users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
        users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
        users_df = users_df.rename_axis(None, axis=1).fillna(0)
        users_df = users_df.add_prefix('user_')
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
                
        # Prepare item features
        
        # Transform genres into binary values
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        # Normalize the values so that each movie's genres sum up to 1
        
        interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
            / interactions_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Prepare input data and fit the model
    
        # Dot product for personalization
        if self.uses_dot_product:
            interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
                * interactions_df[self.user_features].values
        
        if self.uses_dot_product:  # Personalized
            x = interactions_df.loc[:, self.mlb.classes_].values
        else:  # Non-personalized
            x = interactions_df.loc[:, list(self.mlb.classes_) + self.user_features].values
        
        y = interactions_df['rating'].values
    
        self.model = SVR(C=self.c, epsilon=self.epsilon).fit(x, y)

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns
        top n_recommendations for each user.

        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations
            should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations
            for each user.
        :rtype: pd.DataFrame
        """

        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        items_df[self.mlb.classes_] = items_df[self.mlb.classes_] \
            / items_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
                user_df.columns = self.user_features
            input_df = items_df.copy()
            
            if self.uses_dot_product:
                input_df[self.mlb.classes_] = items_df[self.mlb.classes_] * user_df.values
                scores = self.model.predict(input_df.loc[:, self.mlb.classes_].values)
            else:
                input_df = input_df.merge(user_df, how='cross')
                scores = self.model.predict(input_df.loc[:, list(self.mlb.classes_) + self.user_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

    @staticmethod
    def _transform_genres(df):
        """
        Transforms a string with genres into a list of cleaned genre names.

        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df
    
    
# Quick test of the recommender

svr_recommender = SVRRecommender(C = 1.7662828959440695, epsilon = 0.4211956483110043)
svr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = svr_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,147286,4.619899,The Adventures of Sherlock Holmes and Doctor Watson: The Treasures of Agra (1983),Crime|Mystery
1,1,146028,4.619899,The Adventures of Sherlock Holmes and Dr. Watson: The Hound of the Baskervilles (1981),Crime|Mystery
2,1,2921,4.422932,High Plains Drifter (1973),Western
3,1,50,4.392314,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
4,1,930,4.371783,Notorious (1946),Film-Noir|Romance|Thriller
5,1,4349,4.332937,Catch-22 (1970),Comedy|War
6,1,750,4.332937,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Comedy|War
7,1,73323,4.321873,"Girl Who Kicked the Hornet's Nest, The (Luftslottet som sprängdes) (2009)",Action|Crime|Mystery
8,1,5288,4.181632,"Atomic Cafe, The (1982)",Documentary|War
9,1,42556,4.164963,7 Faces of Dr. Lao (1964),Fantasy|Mystery|Western


In [6]:
svr_recommender = SVRRecommender(C = 1.7662828959440695, epsilon = 0.4211956483110043)

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

,Recommender,RMSE,MRE,TRE
0,SVRRecommender,0.9557,0.327759,0.213112


In [7]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

# Split into train_validation and test sets

shuffle = np.arange(len(ml_ratings_df))
rng.shuffle(shuffle)
shuffle = list(shuffle)

train_test_split = 0.8
split_index = int(len(ml_ratings_df) * train_test_split)

train_validation = ml_ratings_df.iloc[shuffle[:split_index]].loc[:, ['user_id', 'item_id', 'rating']]
test = ml_ratings_df.iloc[shuffle[split_index:]].loc[:, ['user_id', 'item_id', 'rating']]

# Tune

def loss(tuned_params):
    svr_recommender = SVRRecommender(kernel='rbf', C=tuned_params['C'], epsilon=tuned_params['epsilon'])
    rmse, mre, tre = evaluate_train_test_split_explicit(
        svr_recommender, train_validation, ml_movies_df, seed=seed)
    return rmse

# Define the search space for hyperopt
# Write your code here

space = {
    'C': hp.uniform('C', 0, 10),
    'epsilon': hp.uniform('epsilon', 0, 1)
}

succeded = False
n_tries = 3
t = 0
while not succeded and t < n_tries:
    try:
        trials = Trials()
        best_param_set = fmin(loss, space=space, algo=tpe.suggest, max_evals=10, show_progressbar=True, trials=trials)
        succeded = True
    except:
        t += 1
        traceback.print_exc()

# Best params

print("C = {}".format(best_param_set['C']))
print("epsilon = {}".format(best_param_set['epsilon']))
    
# Test

C = 0.8078294657404328
epsilon = 0.3898720771476633

svr_recommender = SVRRecommender(C = best_param_set['C'], epsilon = best_param_set['epsilon'])

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, {'train': train_validation, 'test': test}, ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

100%|██████████| 10/10 [02:51<00:00, 17.14s/trial, best loss: 0.9907539408896741]
C = 1.7251088776592127
epsilon = 0.27240368669508286


,Recommender,RMSE,MRE,TRE
0,SVRRecommender,0.946341,0.326655,0.212039


**Task 7.** Gather the results for LinearRegressionRecommender, SVRRecommender, TFIDFRecommender, MostPopularRecommender, HighestRatedRecommender, RandomRecommender from the evaluate_train_test_split_implicit method and print them as a single DataFrame.

In [49]:
linearRegressionRecommender = LinearRegressionRecommender()
svrRecommender = SVRRecommender()
tfidfRecommender = TFIDFRecommender()
mostPopularRecommender = MostPopularRecommender()
highestRatedRecommender = HighestRatedRecommender()
randomRecommender = RandomRecommender()

recommenders = [linearRegressionRecommender, svrRecommender, tfidfRecommender, mostPopularRecommender, highestRatedRecommender, randomRecommender]

cols = ['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10']

results = []

for recommender in recommenders:
    
    result = evaluate_train_test_split_implicit(recommender, ml_ratings_df, ml_movies_df)
    result = [type(recommender).__name__] + list(result)
    result = pd.DataFrame([result], columns=cols)
    results.append(result)
    
results = pd.concat(results).reset_index(drop=True)
display(results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionRecommender,0.000000,0.006711,0.015660,0.080537,0.000000,0.003942,0.007501,0.028281
1,MostPopularRecommender,0.076063,0.284116,0.440716,0.680089,0.076063,0.192392,0.256306,0.332798
2,HighestRatedRecommender,0.114094,0.239374,0.331096,0.568233,0.114094,0.186693,0.224823,0.302199
3,RandomRecommender,0.000000,0.008949,0.011186,0.035794,0.000000,0.005353,0.006316,0.013998
